In [1]:
%load_ext autotime

In [2]:
import os
from glob import glob

from astropy.io import fits
from astropy.stats import sigma_clipped_stats
import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
plt.style.use('bmh')

from tqdm import tqdm_notebook

from piaa.utils import pipeline
from piaa.utils.postgres import get_cursor
from piaa.utils import helpers
from pocs.utils.images import fits as fits_utils
from pocs.utils.images import crop_data

from piaa.utils.postgres import get_cursor
tess_cursor = get_cursor(port=5433, db_name='v6', db_user='postgres')

KeyboardInterrupt: 

time: 32.3 s


In [ ]:
# base_dir = '/var/panoptes/images/fields/PAN001/Hd189733/14d3bd/20180913T085704/'
base_dir = '/var/panoptes/images/fields/PAN012/358d0f/20180822T035809/'

In [ ]:
sequence = 'PAN012_358d0f_20180822T035809'

In [ ]:
# Get all the files that start with a time stamp (skip pointing files)
fits_files = sorted(glob(
    os.path.join(base_dir, '*2018*.fits')
), )
print("Found {} FITS files".format(len(fits_files)))

## Lookup point sources

Looking up the point sources invovles two steps, extracting detected sources from the image and matching those with a catalog.

> Note: This assumes the images have already been downloaded and plate-solved.  
> _todo(wtgee) add a link to a notebook that does that_

**Source Detection** For source extraction there are three methods available but the default is to use the well-known [`sextractor`](https://www.astromatic.net/software/sextractor) to  automatically detect sources according to certain configurable parameters. The [PIAA sextractor config](https://raw.githubusercontent.com/panoptes/PIAA/develop/resources/conf_files/sextractor/panoptes.sex) sets a photometric aperture of 6 pixels ($6 \times 10.3'' \approx 80''$) and a detection threshold of 1.5 sigmas. Further filtering of flags and basic SNR is done below.

**Catalog Matching** PANOPTES hosts a copy of the TESS Input Catalog (v6) against which the detected sources are matched. The detection uses the peak pixel value as reported by `sextractor` compared against the TICv6 using standard [astropy catalog matching](http://docs.astropy.org/en/stable/coordinates/matchsep.html#matching-catalogs) techniques. Reported catalog separation is neglible, especially regarding the creationg of the stamps for the RGB pixel pattern (see below).

In [ ]:
# Lookup the point sources
point_sources = pipeline.lookup_point_sources(
    fits_files[0], 
    force_new=True,
    cursor=tess_cursor,
)

# Display
display(point_sources.head())
print(f'Sources extracted: {len(point_sources)}')

#### Lookup all files

Perform the same thing again but for all files. The resulting source catalog created from `sextractor` is saved for each image and reused in subsequent runs unless `force_new=True`.

Extract the observation time, the exposure time, and the airmass from the FITS headers.

The image observation time currently comes from the image file name, which is generated by the computer running the unit and is given in UTC. This is mainly due to a problem with the `DATE-OBS` header in the FITS files for POCS versions <0.6.2.

In [ ]:
source_filename = os.path.join(base_dir, f'point-sources-detected.csv')

observation_sources = pipeline.lookup_sources_for_observation(
    fits_files=fits_files, 
    filename=source_filename, 
    cursor=tess_cursor, 
    force_new=True
)

num_sources = len(set(observation_sources.id.values))
print(f"Num sources: {num_sources}")

#### Image filtering

We can do some quick checks on the overall image sequence:

In [ ]:
image_group = observation_sources.groupby('obs_time')

In [ ]:
image_group.snr.mean().plot()
plt.title('Observation Mean SNR')

In [ ]:
plt.figure(figsize=(6, 4.5))
image_group.background.mean().plot()
plt.title(f'Background Mean')

The two plots above tell us that there are some changing conditions (e.g. clouds) toward the last part of the observation. These frames are manually removed from the sequence:

In [ ]:
observation_sources = observation_sources.loc[:'2018-08-22 07:06:00']

In [ ]:
num_sources = len(set(observation_sources.id))
print(f"Num sources: {num_sources}")

#### Source filtering

`sextractor` does basic thresholding and source detection per frame but here we filter sources based off the entire image sequence. The simplest filter is to get all the images with a certain SNR limit (default 10). A filter for source extractor flags can also be used.

> Note: There has been a manual selection of images (see above) for overall data quality. Some kind of image quality check (e.g. drifting zeropoint) could help automate this

In [ ]:
num_sources = len(set(observation_sources.id))
print(f"Num sources: {num_sources}")

In [ ]:
id_group = observation_sources.groupby('id')

##### sextractor flags filter

Filter out anything with [SE flags](http://matilda.physics.ucdavis.edu/working/website/SEflags.html) of 4 or more. Note that this is leaving in some blended sources 

In [ ]:
observation_sources = id_group.filter(lambda grp: any(grp['snr'] >= 4))

num_sources = len(set(observation_sources.id))
print(f"Num sources: {num_sources}")

##### SNR Filter

The SNR comes directly from `sextractor` in the form of `FLUX_AUTO / FLUXERR_AUTO`.

In [ ]:
snr_limit = 10

observation_sources = id_group.filter(lambda grp: grp['snr'].mean() > snr_limit)

num_sources = len(set(observation_sources.id))
print(f"Num sources: {num_sources}")

#### Output filtered source detections

We save this filtered file to be used for subsequent processing.

In [ ]:
filtered_filename = os.path.join(base_dir, f'point-sources-filtered.csv')

In [ ]:
observation_sources.to_csv(filtered_filename)